In [2]:
import os
import math
import glob
import cv2
from collections import deque
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm
from moviepy.editor import VideoFileClip
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from scipy.ndimage.measurements import label
import pickle

In [3]:
def extract_hog_features(image):
    win_size = (64, 64)
    block_size = (16, 16)
    block_stride = (8, 8)
    cell_size = (8, 8)
    nbins = 9

    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    features = hog.compute(image)
    return features.flatten()

# X = []
# y = []

# # Assuming you have a 'dataset' directory with 'vehicles' and 'non-vehicles' subdirectories
# base_dir = 'dataset'

# # Define labels for vehicles and non-vehicles
# vehicle_label = 'vehicle'
# non_vehicle_label = 'non-vehicle'

# # Iterate through the subdirectories under 'vehicles' and 'non-vehicles'
# for category in [vehicle_label, non_vehicle_label]:
#     category_dir = os.path.join(base_dir, category)
    
#     for root, dirs, files in os.walk(category_dir):
        
#         for filename in files:
#             print(filename)
#             break
#             if filename.endswith('.jpg') or filename.endswith('.png'):
#                 image_path = os.path.join(root, filename)
#                 image = cv2.imread(image_path)
#                 if image is not None:
#                     features = extract_hog_features(image)
#                     X.append(features)
#                     y.append(category)

# print(X)


In [4]:
test_images = np.array([plt.imread(i) for i in glob.glob('./test_images/*.jpg')])

car_images = []
non_car_images = []

for root, dirs, files in os.walk('./dataset/vehicles/'):
    for file in files:
        if file.endswith('.png'):
            car_images.append(os.path.join(root, file))
            
for root, dirs, files in os.walk('./dataset/non-vehicles/'):
    for file in files:
        if file.endswith('.png'):
            non_car_images.append(os.path.join(root, file))

In [5]:
X = []
y = []
for img in car_images:
    img = cv2.imread(img)
    X.append(extract_hog_features(img))
    y.append(1)
    
for img in non_car_images:
    img = cv2.imread(img)
    X.append(extract_hog_features(img))
    y.append(0)
# print(X,"\n", y)
    

In [6]:
from sklearn import svm
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(kernel='linear')

In [7]:
print(clf)

SVC(kernel='linear')


In [8]:
video_capture = cv2.VideoCapture('test_video.mp4')

while video_capture.isOpened():
    ret, frame = video_capture.read()
#     print(ret, frame)
    if not ret:
        break
# frame = cv2.resize(frame, (640, 480))  # Resize the frame to a fixed size
# gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
#     # Process the frame (resize, grayscale, etc.)

#     # Apply sliding window technique and extract HOG features for each window
#     # In this part, you loop over different windows in the frame and extract HOG features for each window.
#     # The exact code for this may vary depending on your specific implementation.

#     # Use the trained SVM classifier to classify each window
#     # You can use the `clf.predict()` method to classify each window, like this:

#     for window in windows:
#         hog_features = extract_hog_features(window)
#         prediction = clf.predict([hog_features])

#         if prediction == 1:  # 1 indicates the presence of a car
#             # Here, you've detected a car in the current window.
#             # You can draw a bounding box around the window or perform further actions to indicate the car's presence in the frame.
#             # For example, you can draw a bounding box using OpenCV's cv2.rectangle() function.
#             x, y, w, h = window_coordinates  # Define these coordinates based on your sliding window approach.
#             cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw a green bounding box around the detected car.

    window_size = (64, 64)  # Define the size of your sliding window
    step_size = 16  # Define the step size for window movement

    for y in range(0, frame.shape[0] - window_size[1], step_size):
        for x in range(0, frame.shape[1] - window_size[0], step_size):
            window = frame[y:y+window_size[1], x:x+window_size[0]]  # Extract the window
            # Extract HOG features for this window
            hog_features = extract_hog_features(window)
            # Use the SVM classifier to predict if it contains a car
            prediction = clf.predict([hog_features])
            if prediction == 1:
                # You can draw a bounding box to visualize the detected car
                cv2.rectangle(frame, (x, y), (x+window_size[0], y+window_size[1]), (0, 255, 0), 2)

        cv2.imshow('Car Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

video_capture.release()
cv2.destroyAllWindows()


4.8.1
